# Demo of the CNN model on Amazon Reviews

In [1]:
import pandas as pd
from pathlib import Path
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch_models.preprocess.preprocess import Preprocessor, PreprocessorConfig
from torch_models.tokenizers.sentencepiece import SentencepieceConfig
from torch_models.tokenizers.bert_wordpiece import BERTWordpieceConfig
from torch_models.dataset.dataset import MulitModalDatasetWithTargets, collate_wrapper_with_targets
from torch.nn import MSELoss
from torch.optim import lr_scheduler, Adam
from torch.utils.data import DataLoader
from torch_models.train.train import train_epoch, evaluation
from torch_models.models.base_convolutional_model import BaseConvolutionalModelConfig, BaseConvolutionalModel
CWD = Path.cwd()
DATA_PATH = CWD.parent / "datasets/amazon_reviews.pkl"

c:\Users\grego\Anaconda3\envs\py311\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
c:\Users\grego\Anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_pickle(str(DATA_PATH))
data.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,month,day,n_images
0,1.0,Not a watercolor book! Seems like copies imo.,It is definitely not a watercolor book. The p...,[{'small_image_url': 'https://m.media-amazon.c...,B09BGPFTDB,B09BGPFTDB,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-01-17 06:06:38.485,0,True,January,Monday,16
1,5.0,Updated: after 1st arrived damaged this one is...,Updated: after first book arrived very damaged...,[],0593235657,0593235657,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2021-12-27 18:26:44.904,1,True,December,Monday,0
2,5.0,Excellent! I love it!,I bought it for the bag on the front so it pai...,[],1782490671,1782490671,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2021-12-24 22:04:55.102,0,True,December,Friday,0
3,5.0,Updated after 1st arrived damaged. Excellent,Updated: after 1st arrived damaged the replace...,[],0593138228,0593138228,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2021-12-24 16:55:06.602,0,False,December,Friday,0
4,5.0,Beautiful patterns!,I love this book! The patterns are lovely. I ...,[{'small_image_url': 'https://m.media-amazon.c...,0823098079,0823098079,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2021-11-19 08:57:33.230,0,True,November,Friday,4


In [4]:
train, test = train_test_split(data, test_size = 0.3, random_state=42)
test, val = train_test_split(test, test_size = 0.5, random_state=42)

In [5]:
val.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,month,day,n_images
81410,5.0,Enjoying a Fresh Brewed LIfe,Fresh-Brewed Life:A Stirring Invitation to Wak...,[],1400203155,1400203155,AH2V2363PPQ3T4AQTLTLMVW26DJA,2011-09-05 02:45:08.000,0,False,September,Monday,0
71182,4.0,Very intense story,This book was nothing like I thought it would ...,[],B07XCSPBKR,B07XCSPBKR,AHWCZ47A7FIYEJ7KEJG3BO3F5YXQ,2022-04-08 15:25:15.220,0,False,April,Friday,0
44102,5.0,"📚 Beautifully written, creepy and evocative, a...","A Distant Grave is beautifully written, super ...",[],1250256445,1250256445,AHTLG4HMRAKA2PV64QS2LL2RSOHA,2021-06-22 11:33:16.479,0,False,June,Tuesday,0
69451,5.0,Very Nice,A fun and colorful book and an easy read,[],0960021930,0960021930,AGCYSGOBKNTIDQ25NKKBP5DRY7YA,2022-05-24 01:44:07.735,0,True,May,Tuesday,0
78508,4.0,Fighting Fear,A little girl relies on an imaginary friend to...,[{'small_image_url': 'https://images-na.ssl-im...,1328470385,1328470385,AH665SQ6SQF6DXAGYIQFCX76LALA,2020-02-29 09:49:32.604,0,False,February,Saturday,2


In [6]:
tokenizer_params = {
    'title': {
        'tokenizer_type': 'BERTWordpiece',
        'config': BERTWordpieceConfig(model_max_length=256)
    },
    'text': {
        'tokenizer_type': 'BERTWordpiece',
        'config': BERTWordpieceConfig(model_max_length=256)
    }
}


preprocessor_config = PreprocessorConfig(   
    numeric_cols=['helpful_vote','n_images'],
    categorical_cols=['verified_purchase','month','day'],
    tokenizer_params=tokenizer_params)

preprocessor = Preprocessor(preprocessor_config)

preprocessor.fit(train)

In [7]:
train_preprocessed = preprocessor.transform(train)
test_preprocessed = preprocessor.transform(test)

In [8]:
train_preprocessed["loss_weights"] = np.ones(train_preprocessed['numeric'].shape[0])
train_preprocessed["target"] = train["rating"].values

test_preprocessed["loss_weights"] = np.ones(test_preprocessed['numeric'].shape[0])
test_preprocessed["target"] = test["rating"].values

In [9]:
train_preprocessed

{'numeric': array([[-0.14872801, -0.11150592],
        [-0.14872801, -0.11150592],
        [-0.09214732, -0.11150592],
        ...,
        [-0.14872801, -0.11150592],
        [ 0.07759477, -0.11150592],
        [-0.09214732, -0.11150592]]),
 'categorical': array([[1., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]),
 'text': {'title': {'input_ids': tensor([[  1, 834,   0,  ...,   0,   0,   0],
          [  1, 213, 414,  ...,   0,   0,   0],
          [  1,  40, 560,  ...,   0,   0,   0],
          ...,
          [  1,  23, 201,  ...,   0,   0,   0],
          [  1, 218, 206,  ...,   0,   0,   0],
          [  1,  40, 714,  ...,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,

In [10]:
train_dataset = MulitModalDatasetWithTargets(train_preprocessed, dtype=torch.float64)
test_dataset = MulitModalDatasetWithTargets(test_preprocessed, dtype=torch.float64)

In [11]:
train_dataset[0]

(tensor([-0.1487, -0.1115], dtype=torch.float64),
 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 0.], dtype=torch.float64),
 [tensor([  1, 834,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0, 

In [12]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce MX250


In [13]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_wrapper_with_targets,
    #pin_memory=True,
    #pin_memory_device=device
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_wrapper_with_targets,
    #pin_memory=True,
    #pin_memory_device=device
    )

In [14]:
basic_conv_config = BaseConvolutionalModelConfig(
    numeric_cols=preprocessor.numeric_pipeline.get_feature_names_out().tolist(),
    categorical_cols=preprocessor.categorical_pipeline.get_feature_names_out().tolist(),
    text_cols=['title','text'],
    #text_cols = ['title'],
    #text_cols=[],
    text_token_cardinalities=(1000,1000),
    text_padding_index=(0,0),
    text_cols_max_tokens=(256,256),
    text_embedding_dim=8,
    text_convolution_filter_sizes=(3,6),
    text_convolution_num_filters=(20,20),
    hidden_layer_dims = (64,1)
)

base_model = BaseConvolutionalModel(basic_conv_config)
base_model = base_model.to(device=device)

In [15]:
sum(p.numel() for p in base_model.parameters() if p.requires_grad)

23925

In [16]:
loss_fn = MSELoss()
optimizer = Adam(
    base_model.parameters(), lr=0.0005)
scheduler = lr_scheduler.LinearLR(
    optimizer, start_factor=1.0, end_factor=0.5, total_iters=30)

In [17]:
eval_losses = []
patience = 3
MODELS_PATH = CWD.parent / "datasets/amazon_reviews_model.model"
for epoch in range(20):
    mean_train_loss, time_elapsed = train_epoch(
        base_model, train_dataloader, loss_fn, optimizer, device, scheduler)
    print(f"Training Epoch: {epoch}")
    print(f"Training mean loss: {mean_train_loss:^8.3f}")
    print(f"Training time elapsed: {time_elapsed:^8.2f}")
    eval_loss, time_elapsed = evaluation(base_model, test_dataloader, loss_fn, device) 
    print(f"Test mean loss: {eval_loss:^8.3f}")
    print(f"Test time elapsed: {time_elapsed:^8.2f}")
    eval_losses.append(eval_loss)
    if epoch == 0: min_eval_loss = eval_loss
    else: min_eval_loss = min(min_eval_loss, eval_loss)
    if eval_loss == min_eval_loss:
        torch.save(base_model.state_dict(), MODELS_PATH)
    if len(eval_losses) > patience:
        if not any([eval_loss == l for l in eval_losses[-patience:]]):
            print("No improvement on test data, stopping training")
            break

base_model.load_state_dict(torch.load(MODELS_PATH))

100%|██████████| 2188/2188 [01:09<00:00, 31.60it/s]


Training Epoch: 0
Training mean loss:  1.412  
Training time elapsed:  69.27  


100%|██████████| 469/469 [00:08<00:00, 53.04it/s]


Test mean loss:  0.819  
Test time elapsed:   8.87  


100%|██████████| 2188/2188 [01:10<00:00, 30.91it/s]


...

100%|██████████| 2188/2188 [01:23<00:00, 26.12it/s]


Training Epoch: 19
Training mean loss:  0.422  
Training time elapsed:  83.78  


100%|██████████| 469/469 [00:10<00:00, 45.63it/s]
C:\Users\grego\AppData\Local\Temp\ipykernel_11940\3609791826.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_mode

Test mean loss:  0.547  
Test time elapsed:  10.31  


<All keys matched successfully>

In [18]:
val_preprocessed = preprocessor.transform(val)
val_preprocessed["loss_weights"] = np.ones(val_preprocessed['numeric'].shape[0])
val_preprocessed["target"] = val["rating"].values
val_dataset = MulitModalDatasetWithTargets(val_preprocessed, dtype=torch.float64)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=1000,
    shuffle=False,
    collate_fn=collate_wrapper_with_targets,
    #pin_memory=True,
    #pin_memory_device=device
    )

In [19]:
base_model.eval()
val_preds, val_actuals = [], []
for batch in val_dataloader:
    numeric = batch.numeric.to(device)
    categorical = batch.categorical.to(device)
    text = [text.to(device) for text in batch.text]
    targets = batch.target.to(device)
    val_actuals.append(targets)
    loss_weights = batch.loss_weights.to(device)

    with torch.no_grad():
        preds = base_model(numeric, categorical, text)
        val_preds.append(preds)

In [20]:
val_preds

[tensor([[4.3558],
         [3.7033],
         [5.0739],
         [5.0142],
         [3.9405],
         [4.8346],
         [4.6911],
         [4.3902],
         [4.8924],
         [4.8167],
         [4.2244],
         [3.3029],
         [4.2479],
         [3.2375],
         [3.5359],
         [4.6329],
         [4.5165],
         [3.5302],
         [4.7315],
         [4.4716],
         [4.8981],
         [3.8994],
         [2.2399],
         [4.7486],
         [5.1533],
         [5.0056],
         [4.5295],
         [4.1422],
         [5.1946],
         [4.7652],
         [4.7544],
         [4.3258],
         [4.4701],
         [4.8961],
         [4.5367],
         [5.1326],
         [4.8580],
         [4.1876],
         [4.7535],
         [4.4918],
         [4.9663],
         [4.4438],
         [4.9005],
         [3.4398],
         [4.7277],
         [2.6194],
         [4.1112],
         [3.0939],
         [3.8217],
         [3.8936],
         [4.6492],
         [3.1685],
         [3.

In [22]:
import numpy as np
v_preds = np.concatenate([v.cpu().detach().numpy() for v in val_preds]).reshape(-1)
v_actuals = np.concatenate([v.cpu().detach().numpy() for v in val_actuals]).reshape(-1)
val_results = pd.DataFrame({'actual':v_actuals,'pred':v_preds})

In [23]:
val_results.groupby('actual').agg(['mean','count'])

pred      
            mean count
actual                
1.0     2.939744   401
2.0     3.112644   548
3.0     3.594306  1532
4.0     4.224787  3659
5.0     4.691731  8860

In [ ]:
base_model.eval()
train_preds, train_actuals = [], []
for batch in train_dataloader:
    numeric = batch.numeric.to(device)
    categorical = batch.categorical.to(device)
    text = [text.to(device) for text in batch.text]
    targets = batch.target.to(device)
    train_actuals.append(targets)
    loss_weights = batch.loss_weights.to(device)

    with torch.no_grad():
        preds = base_model(numeric, categorical, text)
        train_preds.append(preds)

In [ ]:
import numpy as np
t_preds = np.concatenate([v.detach().numpy() for v in train_preds]).reshape(-1)
t_actuals = np.concatenate([v.detach().numpy() for v in train_actuals]).reshape(-1)
train_results = pd.DataFrame({'actual':t_actuals,'pred':t_preds})

In [ ]:
train_results.groupby('actual').agg(['mean','count'])

pred       
            mean  count
actual                 
1.0     2.281060   1961
2.0     2.741797   2532
3.0     3.529326   7058
4.0     4.239303  16648
5.0     4.660447  41801

: 

In [ ]:
train_dataset[0]

(tensor([-0.1487, -0.1115], dtype=torch.float64),
 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 0.], dtype=torch.float64),
 [tensor([  1, 836,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0, 

In [ ]:
train_dataset[1]

(tensor([-0.1487, -0.1115], dtype=torch.float64),
 tensor([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0.], dtype=torch.float64),
 [tensor([  1, 215, 416, 940, 660, 331, 198, 544, 474, 650, 647, 166, 196,  42,
          429,  10,  48, 172,  11,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0, 

In [ ]:
val_dataloader2 = DataLoader(
    val_dataset,
    batch_size=5,
    shuffle=False,
    collate_fn=collate_wrapper_with_targets,
    #pin_memory=True,
    #pin_memory_device=device
    )
d = next(iter(val_dataloader2))

In [ ]:
base_model.eval()
numeric,categorical,text = base_model.basic_input_layer(d.numeric, d.categorical, d.text)

In [ ]:
from torch.nn import functional as F
if len(base_model.config.text_cols) > 0:
    text_conv_list = []
    for i, t in enumerate(text):
        t = t.permute(0,2,1)
        x_conv_list = [F.leaky_relu(conv1d(t)) for conv1d in base_model.conv1d_list[i]]
        x_pool_list = [F.max_pool1d(x_conv, kernel_size = x_conv.shape[2])
            for x_conv in x_conv_list]
        x = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list], dim=1)
        text_conv_list.append(base_model.text_output_layers[i](x))
items_to_concat = []
if len(base_model.config.numeric_cols) > 0:
    items_to_concat.append(numeric)
if len(base_model.config.categorical_cols) > 0:
    items_to_concat.append(categorical)
if len(base_model.config.text_cols) > 0:
    items_to_concat += text_conv_list
t = torch.cat(items_to_concat, dim=1)

In [ ]:
t

tensor([[-1.3506, -1.7846,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.3201,
         -0.9035, -0.0099, -1.3807, -0.6966, -0.3037, -0.3306, -0.1387, -0.3349,
         -0.3046, -0.0264, -0.1481,  0.0698,  0.1269, -0.1963, -0.0191,  3.0223,
         -5.8530, -2.2765, -2.2766,  0.6525, -2.3117,  2.8114, -5.9580, -5.3217,
         -5.6816,  2.9445, -5.4314, -2.4385, -2.6797, -5.8081, -2.8868],
        [-1.3506, -1.7846,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.3070,
         -0.8902, -0.0083, -1.3673, -0.6823, -0.2885, -0.3170, -0.1288, -0.3212,
         -0.2898, -0.0151, -0.1618,  0.0807,  0.1141, -0.1872, -0.0157,  2.2893,
         -5.2921, -1.7156, -1.7174, 

In [ ]:
for i in range(len(base_model.config.hidden_layer_dims)):
    t = F.leaky_relu(t)
    #t = base_model.dropout_list[i](t)
    t = base_model.hidden_layers[i](t)
    print(t)

tensor([[-19.6936, -11.8101, -11.5290, -21.7912, -22.4660, -22.8542, -21.2706,
         -20.6235, -22.7654, -21.2217, -19.2178, -14.5939, -22.8737, -24.2154,
         -22.5567, -22.3282, -18.6685, -13.0706,  -9.8854, -11.2388, -15.7119,
         -12.8468, -21.1208,  -6.4421,  -9.7723,  -7.8745, -21.9087, -22.7294,
         -22.1450, -23.2949, -13.9484, -13.2680, -15.1681,  -7.6689, -22.2565,
         -22.9878, -22.5190, -15.1622, -22.2130, -17.9637, -22.9702, -12.4932,
         -20.5140, -22.5401, -17.9717, -21.4823, -12.4711,  -6.4177, -22.6987,
         -23.1636, -23.0665, -22.3570, -22.6676, -20.2828, -22.7712,  -7.4171,
         -12.7370, -22.1104,  -8.3587, -22.7795, -22.5576, -11.3994, -22.5609,
         -14.8635],
        [-17.4355, -10.8803, -11.1955, -18.0318, -18.4790, -19.0045, -19.3583,
         -18.0495, -19.1670, -18.3461, -16.8999, -13.0117, -19.0227, -20.4707,
         -18.7250, -18.3994, -17.2311, -10.0465, -10.5677, -11.5728, -14.7947,
         -12.4634, -18.7844,  -5

In [40]:
t

tensor([[3.5546],
        [2.5133],
        [6.2138],
        [3.1018],
        [5.0069]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [26]:
pd.Series(preprocessor.tokenizers['text'].fast_tokenizer.get_vocab()).sort_values()

[PAD]           0
[CLS]           1
[UNK]           2
!               3
"               4
             ... 
toget         995
alth          996
everything    997
class         998
ins           999
Length: 1000, dtype: int64